In [1]:
%matplotlib inline

In [2]:
import pymysql
import pandas as pd
import getpass
from textblob import TextBlob
import re

In [7]:
conn = pymysql.connect(host="jupyter.med.utah.edu",
                       port=6033,user="jovyan",
                       passwd=getpass.getpass("Enter MySQL passwd for jovyan"),db='mimic2')
cursor = conn.cursor()

Enter MySQL passwd for jovyan········


### Use Pandas and SQL to create a dataframe with the following:
* subject_id
* hospital admission id
* text of the radiology report
* Limit the number of reports to 10000

In [4]:
rad_data = \
pd.read_sql("""SELECT noteevents.subject_id, 
                      noteevents.hadm_id,
                      noteevents.text 
               FROM noteevents
               WHERE noteevents.category = 'RADIOLOGY_REPORT' LIMIT 10000""",conn)
rad_data.head(5)

,subject_id,hadm_id,text
0,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:53 AM\n ...
1,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:53 AM\n ...
2,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:43 AM\n ...
3,56,28766.0,\n\n\n DATE: [**2644-1-17**] 6:37 AM\n ...
4,56,28766.0,\n\n\n DATE: [**2644-1-19**] 12:09 PM\n ...


In [5]:
rad_data.shape

(10000, 3)

In [7]:
print(rad_data.iloc[20]["text"])




     DATE: [**3352-7-6**] 10:20 AM
     CHEST (PORTABLE AP)                                             Clip # [**Clip Number (Radiology) 1631**]
     Reason: fever- eval for infection
     Admitting Diagnosis: NON-HODGKIN'S LYMPHOMIA\BONE MARROW TRANSPLANT
     ______________________________________________________________________________
     UNDERLYING MEDICAL CONDITION:
      54 year old man with T-cell lymphoma, neutropenia
     REASON FOR THIS EXAMINATION:
      fever- eval for infection
     ______________________________________________________________________________
                                     FINAL REPORT
     INDICATION:  Neutropenia and fever.
     
     COMPARISON: [**6-23**], [**6-6**] and [**3352-6-3**].
     
     CHEST AP PORTABLE RADIOGRAPH:  The cardiac, mediastinal and hilar contours are
     unremarkable.  The left lung is clear.  There is a somewhat well circumscribed
     opacity immediately overlying the catheter which runs along the lateral right
 

### Write a function that returns the impression section of a report

#### Hints

* Not every report will have an impression section
* "INTERPRETATION" and "CONCLUSIONS" might be synonyms for "IMPRESSION"
* If an impression section is not found, return the entire report.

In [1]:
def get_impression(report):
    pass    


#### In how many reports did we find an impression section?

In [10]:
sum([count_impression(report) for report in rad_data["text"]])

8189

### Define Regular expressions for data cleansing

* Write a regular expression to replace dates in the reports with ``[**DATE**]``
* Write a regular expression to replace times in the reports with ``[**TIME**]``
* Write a regular expression to replaces digits with "d", (e.g. "43 cc" would become "dd cm")

#### Hints

* Look at some sample reports to see what dates and times look like in the reports
* What order would you need to apply the regular expressions?

In [20]:
for i in range(10):
    print(rad_data.iloc[i]['text'])
    print("*"*42,"\n\n")




     DATE: [**2644-1-17**] 10:53 AM
     MR HEAD W & W/O CONTRAST; MR CONTRAST GADOLIN                   Clip # [**Clip Number (Radiology) 12569**]
     Reason: R ICB and HX brain mets - eval - also with DWI for CVA Do MR
      Contrast: MAGNEVIST Amt: 15
     ______________________________________________________________________________
     UNDERLYING MEDICAL CONDITION:
      [**Age over 90 **] year old woman with lung CA- mets to brain                                   
     REASON FOR THIS EXAMINATION:
      R ICB and HX brain mets - eval - also with DWI for CVA Do MRI both with and 
      without contast please
     ______________________________________________________________________________
                                     FINAL REPORT
     EXAMINATION:  MRI of the brain with and without gadolinium.
     
     INDICATION:  [**Age over 90 **] year old woman with lung cancer and right intracranial bleed
     and history of brain metastases.  Please evaluate for acute infar

In [11]:
date = re.compile(r"""\d""")

In [ ]:
print(rad_data.iloc[0]["text"])

In [12]:
print(rad_data.iloc[0]["text"])




     DATE: [**2644-1-17**] 10:53 AM
     MR HEAD W & W/O CONTRAST; MR CONTRAST GADOLIN                   Clip # [**Clip Number (Radiology) 12569**]
     Reason: R ICB and HX brain mets - eval - also with DWI for CVA Do MR
      Contrast: MAGNEVIST Amt: 15
     ______________________________________________________________________________
     UNDERLYING MEDICAL CONDITION:
      [**Age over 90 **] year old woman with lung CA- mets to brain                                   
     REASON FOR THIS EXAMINATION:
      R ICB and HX brain mets - eval - also with DWI for CVA Do MRI both with and 
      without contast please
     ______________________________________________________________________________
                                     FINAL REPORT
     EXAMINATION:  MRI of the brain with and without gadolinium.
     
     INDICATION:  [**Age over 90 **] year old woman with lung cancer and right intracranial bleed
     and history of brain metastases.  Please evaluate for acute infar

In [13]:
digits = re.compile(r"""\d""")

In [14]:
print(digits.sub("d", rad_data.iloc[0]["text"]))




     DATE: [**dddd-d-dd**] dd:dd AM
     MR HEAD W & W/O CONTRAST; MR CONTRAST GADOLIN                   Clip # [**Clip Number (Radiology) ddddd**]
     Reason: R ICB and HX brain mets - eval - also with DWI for CVA Do MR
      Contrast: MAGNEVIST Amt: dd
     ______________________________________________________________________________
     UNDERLYING MEDICAL CONDITION:
      [**Age over dd **] year old woman with lung CA- mets to brain                                   
     REASON FOR THIS EXAMINATION:
      R ICB and HX brain mets - eval - also with DWI for CVA Do MRI both with and 
      without contast please
     ______________________________________________________________________________
                                     FINAL REPORT
     EXAMINATION:  MRI of the brain with and without gadolinium.
     
     INDICATION:  [**Age over dd **] year old woman with lung cancer and right intracranial bleed
     and history of brain metastases.  Please evaluate for acute infar

In [15]:
rad_data["impression"] = \
rad_data.apply(lambda row: digits.sub("d", get_impression(row["text"])).lower(), axis=1)

In [16]:
rad_data.head()

,subject_id,hadm_id,text,impression
0,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:53 AM\n ...,\n\n\n date: [**dddd-d-dd**] dd:dd am\n ...
1,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:53 AM\n ...,\n\n\n date: [**dddd-d-dd**] dd:dd am\n ...
2,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:43 AM\n ...,stable appearance of right parietal lobe and...
3,56,28766.0,\n\n\n DATE: [**2644-1-17**] 6:37 AM\n ...,\n \n cardiomegaly and mild chf. naso...
4,56,28766.0,\n\n\n DATE: [**2644-1-19**] 12:09 PM\n ...,\n \n marked improvement in left perih...


In [18]:
print(rad_data.iloc[20]["impression"])


     d)  there is a somewhat well-circumscribed but subtle opacity immediately
     adjacent to the external portion of the catheter overlying the right upper
     lobe.  this may be related to the catheter itself but it is difficult to
     exclude an underlying opacity in this region.  correlation with physical exam
     recommended.




### How many unique words occur in the corpus?

#### Hint

1. Use TextBlob
1. Put all the reports into a single string


In [2]:
#" ".join(rad_data["impression"])

### Are all words equal?

In [20]:
from gensim.parsing.preprocessing import STOPWORDS
STOPWORDS

frozenset({'a',
           'about',
           'above',
           'across',
           'after',
           'afterwards',
           'again',
           'against',
           'all',
           'almost',
           'alone',
           'along',
           'already',
           'also',
           'although',
           'always',
           'am',
           'among',
           'amongst',
           'amoungst',
           'amount',
           'an',
           'and',
           'another',
           'any',
           'anyhow',
           'anyone',
           'anything',
           'anyway',
           'anywhere',
           'are',
           'around',
           'as',
           'at',
           'back',
           'be',
           'became',
           'because',
           'become',
           'becomes',
           'becoming',
           'been',
           'before',
           'beforehand',
           'behind',
           'being',
           'below',
           'beside',
           'besides'

### Should STOPWORDS be the same across domains?

In [26]:
my_stop_words = frozenset(["a", "am", "an", "and", "are", "as", "at", "be", "for", "is", "the", "is", "of", "which", ])

## Create a single string with all the reports

#### Hints, etc.
* Use List Comprehension
* Use string joins
* Iterate over the rows of the data frame

### Define a vector space for the radiology corpus

#### Hints

1. How would you build a corpus from words only occuring more than N times?

### Create a new column named `"impression no stops"` where [stop words](https://en.wikipedia.org/wiki/Stop_words) have been dropped from the impression

* The gensim package has stop words defined (``from gensim.parsing.preprocessing import STOPWORDS``

#### Hints
1. Do you agree with dropping all the stop words?
1. How could we create a new stopwords frozen set absent the terms we wan't to keep (double negative?)
1. You could use a regular expressions substitution or token the report first and operate on the list of words.

In [3]:
rad_data["impression no stops"] = None

NameError: name 'rad_data' is not defined

In [4]:
rad_data.iloc[0]["impression"]

NameError: name 'rad_data' is not defined

In [29]:
rad_data.iloc[0]["impression no stops"]

"date: [**dddd-d-dd**] dd:dd mr head w & w/o contrast; mr contrast gadolin clip # [**clip number (radiology) ddddd**] reason: r icb hx brain mets - eval - also with dwi cva do mr contrast: magnevist amt: dd ______________________________________________________________________________ underlying medical condition: [**age over dd **] year old woman with lung ca- mets to brain reason this examination: r icb hx brain mets - eval - also with dwi cva do mri both with without contast please ______________________________________________________________________________ final report examination: mri brain with without gadolinium. indication: [**age over dd **] year old woman with lung cancer right intracranial bleed history brain metastases. please evaluate acute infarct. technique: multiplanar td td-weighted images brain with gadolinium according to standard departmental protocol. no prior study comparison. findings: on diffusion-weighted images there small area restricted diffusion along fal

In [30]:
unique_impression_words = set(TextBlob(" ".join(rad_data["impression no stops"])).words)


In [31]:
word_map = dict(zip(unique_impression_words,range(len(unique_impression_words))))

In [32]:
len(word_map)

8719

In [33]:
word_map

{'subcm': 4304,
 'tracheomalacia': 0,
 'extent': 4295,
 'ld-sd': 1,
 'neck': 4305,
 'entered': 2,
 'r-sided': 3,
 'nailing': 4,
 'sbr': 5,
 'reclotting': 6,
 'nerves': 4309,
 'but': 7,
 'extrinsic': 4310,
 'symmetry': 8,
 'ischemia/infarct': 4311,
 'neostigmine': 9,
 'communicataed': 10,
 'tube/catheters': 11,
 'satellite': 4312,
 'posteriorly-layering': 12,
 'paraventricular': 13,
 'attributed': 14,
 'gluteal': 15,
 'pneumonia/aspiration': 16,
 'nonlaser': 17,
 'obscuring': 18,
 'promptly': 19,
 'continuedclinical': 655,
 'final': 4315,
 'regimen': 6851,
 'mod': 4316,
 'pooling': 4317,
 'cath/effusions': 20,
 'enlargment.questionable': 23,
 'nondistended': 22,
 'lumbosacral': 4320,
 'saved': 24,
 'wire': 7999,
 'vealuate': 4322,
 'several': 4323,
 'tracheoplasty': 25,
 'reviewing': 26,
 'peelaway': 7749,
 'expansive': 27,
 'adaptation': 29,
 'unusual': 4306,
 'electrolyte': 4325,
 'cradiac': 4308,
 'performance': 4326,
 'architectural': 4328,
 'td-signal': 30,
 'a-fib': 31,
 'spiculat